<a href="https://colab.research.google.com/github/vnewsome/project_voting/blob/main/ML_project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Dear Students,

# To make the choropleth/heat maps from the starter notebook, you have to do a couple of steps on Colab.

# Before the first code chunk, run ! pip3 install mapclassify​ in a chunk. This will install an additional package that geopandas needs.
# From the VA Dept of Emergency Management, get the shapefiles: https://vgin.vdem.virginia.gov/datasets/777890ecdb634d18a02eec604db522c6/about
# From that archive, I uploaded the following files to Colab:
# VirginiaCounty_ClippedToShoreline.cpg
# VirginiaCounty_ClippedToShoreline.dbf
# VirginiaCounty_ClippedToShoreline.prj
# VirginiaCounty_ClippedToShoreline.shp
# VirginiaCounty_ClippedToShoreline.shp.xml
# VirginiaCounty_ClippedToShoreline.shx

# Then the rest of the notebook runs.

# The IPUMS data is in the github repo. What are these data? IPUMS is a microdata service that mostly distributes versions of the Census data and the American Community Survey, but also does a lot of international data. I downloaded a bunch of county level (NHGIS) data that looked interesting or promising. A lot of the files have similar data in each file, but they all have codebooks. You probably want to flip through a few and think about which have interesting covariates for predicting R vs D voting (e.g. education and education*race, income and income*race, etc.).

# There is a link to FRED for macro data (inflation, unemployment, etc.) in the readme. You can build a model and then plug in projections for November. That's another interesting angle to follow up, if you're interested: Uncertainty driven by the uncertainty in macro projections for the next 6 months.